Importing all the things that are used in this notebook.

In [ ]:
from google.colab import drive

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from huggingface_hub import login
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import hf_hub_download
import pandas as pd

!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from tqdm import tqdm
tqdm.pandas()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

### Mounting Google drive, and huggingFace

Mount the google drive so to stores the data after each stage of the experiment.

Also authorize huggingface with a token (to be fetched from huggingface) so it can download the dataset or models from huggingface. Note that, you probably have to accept terms of a dataset or a model before using it.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

hf_token = os.environ.get("HUGGINGFACE_TOKEN")


login(token=hf_token)

### Download the model
You can change the model to any available AutoModelForCausalLLM model on huggingFace, by changing model_id.

In [ ]:

model_id = "Qwen/Qwen2.5-3B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

### Download the dataset, some analysis, and find classification classes

In [ ]:

# Define folder path
data_path = "/home/data"

# Create the folder if it doesn't exist
os.makedirs(data_path, exist_ok=True)



test_file_path = hf_hub_download(repo_id="murathankurfali/ClimateEval",subfolder='exeter/sub_claim', filename="test.csv", repo_type="dataset")
train_file_path = hf_hub_download(repo_id="murathankurfali/ClimateEval",subfolder='exeter/sub_claim', filename="training.csv", repo_type="dataset")
val_file_path = hf_hub_download(repo_id="murathankurfali/ClimateEval",subfolder='exeter/sub_claim', filename="validation.csv", repo_type="dataset")

test_csv = pd.read_csv(test_file_path)
train_csv = pd.read_csv(train_file_path)
val_csv = pd.read_csv(val_file_path)

print(train_csv.columns)
print(train_csv['sub_claim'].value_counts())
print(train_csv['sub_claim_code'].value_counts())
print( list(zip(train_csv['sub_claim_code'], train_csv['sub_claim'])))
print(f'len of codes are: {len(train_csv["sub_claim_code"])} and len of strings are: {len(train_csv["sub_claim"])}')
print(f'len of test is: {test_csv.shape[0]}')


# creating the sorted list of class labels
counts = train_csv["sub_claim_code"].value_counts().sort_index()

code_to_description = train_csv.drop_duplicates("sub_claim_code").set_index("sub_claim_code")["sub_claim"]

combined = [(code, code_to_description[code]) for code, count in counts.items()]

classes_strings = []
# Display sorted result
for code, desc in combined:
    print(f"{code}: {desc}")
    classes_strings.append(f"{code}: {desc}")

print(classes_strings)
claim_codes = train_csv['sub_claim_code'].unique()
print(claim_codes)




test.csv:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

training.csv:   0%|          | 0.00/7.85M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/877k [00:00<?, ?B/s]

Index(['text', 'sub_claim_code', 'sub_claim'], dtype='object')
sub_claim
No claim                                                                                  16302
Climate-related science is unreliable/uncertain/unsound (data, methods & models)           1373
Climate movement is unreliable/alarmist/corrupt                                            1014
It’s natural cycles/variation                                                               788
Climate hasn’t warmed/changed over the last (few) decade(s)                                 483
Extreme weather isn’t increasing/has happened before/isn’t linked to climate change         427
There’s no evidence for greenhouse effect/carbon dioxide driving climate change             339
Species/plants/reefs aren’t showing climate impacts/are benefiting from climate change      337
Ice/permafrost/snow cover isn’t melting                                                     333
Climate policies (mitigation or adaptation) are harmful        

'\nfiles = ["train.csv", "validation.csv", "test.csv"]  # or whatever is present\n\nfor file in files:\n    r = requests.get( \'https://huggingface.co/datasets/murathankurfali/ClimateEval/tree/main/exeter/sub_claim/\'+ file)\n    with open(data_path+file, "wb") as f:\n        f.write(r.content)\n'

### Classify a given text

Given a prompt with example, classes, and query, the classify_text creates the prompt and use the generate_response (which use the LLM to generate the response), and find the class in the generated response.

In [ ]:
# Generate text
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=32)

    input_length = inputs.input_ids.shape[-1]
    generated_tokens = outputs[0][input_length:]

    #response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return response


def classify_text(text, examples = None):

    if examples is None:
        prompt = f"Classify the following text into one of these categories: {', '.join(classes_strings)}.\nText: \"{text}\"\n Classification result is:"
    else:
        prompt = f"Given the examples: {', '.join(examples)}\n Classify the following text into one of these categories: {', '.join(classes_strings)}.\nText: \"{text}\"\n Classification result is:"
    #prompt = f"Classify the following text into one of these categories: {', '.join(classes_strings)}.\nText: \"{text}\"\n Classification result is:"


    response = generate_response(prompt)
    #print(f'the response is:  {response}')
    predicted_code = "Unknown" # Default value

    for code in claim_codes:
        if code.lower() in response.lower():
            predicted_code = code
            break # Assuming only one category should match
    #print(f'The predicted code is: {predicted_code}')
    return predicted_code




print(generate_response('tell me the truth, is this prompt a test?'))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 I'm sorry, but I'm not sure what you mean by "this prompt." Could you please provide more context or information about the prompt you're referring to


### Example prompting and metrics
examplePromptCreator creates a list of prompt string from examples. The generated strings are then used in the classify_text function.

The calculate metrics also call the classification_report of sklearn.

In [ ]:
def examplePromptCreator(rawExamples):
    examples_string= []
    for i, match in enumerate(rawExamples):
        examples_string.append(f" Example text: \"{match['text']}\" classification result is: {match['sub_claim_code']}: {match['sub_claim']} ")
    return examples_string

def calculate_metrics(y_true, y_pred):
    print(classification_report(y_true, y_pred, target_names=sorted(y_true.unique())))


# Performing three stages of experiment
Note that, at the end of each stage, the results are saved in test_csv and stored on google drive. To continue experiment from the middle, you can uncomment the line that reads the csv (last cell  before each stage), and continue the rest.

# First stage: Zero-shot

In [ ]:

test_csv['predicted_sub_claim'] = test_csv['text'].progress_apply(classify_text)

print("Comparing actual and predicted sub_claims (first 5 rows):")
print(test_csv[['text', 'sub_claim', 'predicted_sub_claim']].head())



# Assume you have two columns: y_true and y_pred
y_true = test_csv["sub_claim_code"]      # true labels column
y_pred = test_csv["predicted_sub_claim"]   # predicted labels column

calculate_metrics(y_true, y_pred)

test_csv.to_csv("/content/drive/MyDrive/riseNLP/zeroShotTest.csv", index=False)

100%|██████████| 2904/2904 [38:22<00:00,  1.26it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result

Comparing actual and predicted sub_claims (first 5 rows):
                                                text  \
0  The discussion is far from over. The true mind...   
1  NASA's James Hansen claims extreme weather eve...   
2  Figure 1. Lake Baikal paleoclimate record from...   
3  Nuccitellis answer to this unexceptionable and...   
4  It makes you wonder what created all that CO2 ...   

                                           sub_claim predicted_sub_claim  
0  Climate-related science is unreliable/uncertai...                 1_4  
1                                           No claim                 1_7  
2                                           No claim                 1_1  
3                      It’s natural cycles/variation                 1_1  
4                      It’s natural cycles/variation                 5_1  
              precision    recall  f1-score   support

         0_0       0.89      0.54      0.68      1754
         1_1       0.07      0.96      0.13   

In [ ]:

#test_csv = pd.read_csv("/content/drive/MyDrive/riseNLP/zeroShotTest.csv")

# Second stage: Few shot

In [ ]:
#FEW SHOT PROMPT

unique_examples = test_csv.drop_duplicates(subset="sub_claim_code")

examples= []

for _,row in unique_examples.iterrows():
  examples.append({
        "text": row["text"],
        "sub_claim": row["sub_claim"],
        "sub_claim_code": row["sub_claim_code"] })

random_few_shot_examples = examplePromptCreator(examples)

def classify_text_with_extra_shots_random(text):
    examples = random_few_shot_examples
    predicted_code = classify_text(text, examples = examples)
    return predicted_code


print("Now testing with random few shots!")
test_csv['predicted_sub_claim_random'] = test_csv['text'].progress_apply(classify_text_with_extra_shots_random)

test_csv.to_csv("/content/drive/MyDrive/riseNLP/testwithResultsRandomShots.csv", index=False)


Now testing with random few shots!


100%|██████████| 2904/2904 [1:56:16<00:00,  2.40s/it]


In [ ]:
#test_csv = pd.read_csv("/content/drive/MyDrive/riseNLP/testwithResultsRandomShots.csv")

# Third stage: RAG

In [ ]:
rag_top_k_number= 5


text_encoder_model = SentenceTransformer("all-MiniLM-L6-v2")

text_embeddings = text_encoder_model.encode(train_csv["text"].tolist(), convert_to_tensor=True).cpu().numpy()

def retrieve_top_k_classes(query_text, k=5):
    query_embedding = text_encoder_model.encode([query_text], convert_to_tensor=True).cpu().numpy()

    similarity_scores = cosine_similarity(query_embedding, text_embeddings)[0]

    # Get top-k indices
    top_k_indices = np.argsort(similarity_scores)[-k:][::-1]

    results = []
    for idx in top_k_indices:
        results.append({
            "text": train_csv.loc[idx, "text"],
            "sub_claim_code": train_csv.loc[idx, "sub_claim_code"],
            'sub_claim': train_csv.loc[idx, "sub_claim"],
            "similarity_score": similarity_scores[idx]
        })
    #print(results)
    return results



def classify_text_with_extra_shots_rag(text):
    top_matches = retrieve_top_k_classes(text, k=rag_top_k_number)
    examples = examplePromptCreator(top_matches)
    predicted_code = classify_text(text, examples = examples)
    return predicted_code



test_csv = test_csv.dropna(subset=["text"])

print("Now testing with rag few shots!")
test_csv['predicted_sub_claim_rag'] = test_csv['text'].progress_apply(classify_text_with_extra_shots_rag)

test_csv.to_csv("/content/drive/MyDrive/riseNLP/testwithResultsRag.csv", index=False)


'''
# Example usage
query = "What is the future of renewable energy?"
top_matches = retrieve_top_k_classes(query, k=5)

# Display the results
for i, match in enumerate(top_matches, 3):
    print(f"\nTop {i}")
    print("Matched Text:", match["text"])
    print("Text Class:", match["sub_claim_code"])
    print("Claim:", match["sub_claim"])
    print(f"Similarity Score: {match['similarity_score']:.4f}")

'''




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now testing with rag few shots!


100%|██████████| 2898/2898 [43:19<00:00,  1.11it/s]


'\n# Example usage\nquery = "What is the future of renewable energy?"\ntop_matches = retrieve_top_k_classes(query, k=5)\n\n# Display the results\nfor i, match in enumerate(top_matches, 3):\n    print(f"\nTop {i}")\n    print("Matched Text:", match["text"])\n    print("Text Class:", match["sub_claim_code"])\n    print("Claim:", match["sub_claim"])\n    print(f"Similarity Score: {match[\'similarity_score\']:.4f}")\n\n'

In [ ]:
#test_csv = pd.read_csv("/content/drive/MyDrive/riseNLP/testwithResultsRag.csv")

# Results
Having test_csv with all saved results in their corresponding columns, below we compare all the results.

In [ ]:
print("Results of zero shot!--------------------------------------")
y_true = test_csv["sub_claim_code"]      # true labels column
y_pred = test_csv["predicted_sub_claim"]   # predicted labels column
calculate_metrics(y_true, y_pred)



print("Results of few random shot!--------------------------------------")
y_true = test_csv["sub_claim_code"]      # true labels column
y_pred = test_csv["predicted_sub_claim_random"]   # predicted labels column
calculate_metrics(y_true, y_pred)


print("Results of few shots with RAG!--------------------------------------")
y_true = test_csv["sub_claim_code"]      # true labels column
y_pred = test_csv["predicted_sub_claim_rag"]   # predicted labels column
calculate_metrics(y_true, y_pred)

Results of zero shot!--------------------------------------
              precision    recall  f1-score   support

         0_0       0.90      0.54      0.68      1753
         1_1       0.07      0.96      0.13        51
         1_2       0.50      0.14      0.22        21
         1_3       0.14      0.60      0.23        30
         1_4       0.15      0.44      0.23        68
         1_6       0.33      0.04      0.07        26
         1_7       0.00      0.00      0.00        64
         2_1       0.54      0.10      0.18       124
         2_3       0.00      0.00      0.00        48
         3_1       0.41      0.27      0.33        26
         3_2       0.00      0.00      0.00        49
         3_3       0.00      0.00      0.00        46
         4_1       0.15      0.78      0.25        64
         4_2       0.20      0.06      0.09        34
         4_4       0.00      0.00      0.00        39
         4_5       0.00      0.00      0.00        36
         5_1       0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
